<a href="https://colab.research.google.com/github/mhpbreugem/WP1/blob/main/BBP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [148]:
using Distributions, Random, NLsolve, LinearAlgebra, Printf, Optim

In [237]:
# Chapter 0: Parameters

# Economic parameters
β = 0.95
γ = 4.0
e0 = 3.0
EΠ = 1.0
Π0 = 1.0
θXbar = 1.0
θYbar = 1.0
μθX = 0.0
σθX = 0.2
μθY = 0.0
σθY = 0.1
Nσπ = 2
σΠ = 0.4
ρ = 0

ΞЮ = [repeat([x], 1) for x in range(0.1, stop=1.0, length=10)]

# Grid parameters
Nσπ = 21
NNσπ = 1
Nπ = 2 * Nσπ * NNσπ + 1  # number of grid points for payoff

NσΠ = 2
NNσΠ = 1
NΠ = 2 * NσΠ * NNσΠ + 1  # number of grid points for payoff

NθX = 5
NθY = 5
NσθX = 2
NσθY = 2

# Derived midpoints
μπi = (Nπ + 1) ÷ 2
μΠi = (NΠ + 1) ÷ 2
μθXi = (NθX + 1) ÷ 2

# Total number of states
NS = Nπ
NN = NθX * NθY * NΠ

125

In [150]:
# Chapter 1: Grid Construction

# Conjectured Payoffs
ππ = collect(range(EΠ - Nσπ*σΠ, EΠ + Nσπ*σΠ, length=Nπ))
πω = pdf.(Normal(EΠ, σΠ), ππ)
πΩ = πω ./ sum(πω)

# True Payoffs
ΞΠ = EΠ .+ σΠ .* range(-NσΠ, NσΠ, length=NΠ)
ππ_to_index = Dict(round(ππ[πi], digits=8) => πi for πi in 1:Nπ)
ΞΠi = [ππ_to_index[round(ΞΠ[Πi], digits=8)] for Πi in 1:NΠ]

# Signals
S=ππ

function ψϵω(Ю, ϵ)
    σϵ = 1 / sqrt(Ю[1])
    return pdf(Normal(0.0, σϵ), ϵ)
end

function φSΩπ(Ю)
    σϵ = 1 / sqrt(Ю[1])
    SΩ = [pdf(Normal(π, σϵ), s) for π in ππ, s in ππ]
    SΩnorm = SΩ ./ sum(SΩ, dims=2)
    return SΩnorm
end

# Noise Trader Demand
ΞθX = μθX .+ σθX .* range(-NσθX, NσθX, length=NθX)
ΞθY = μθY .+ σθY .* range(-NσθY, NσθY, length=NθY)

function θXYωω(x, y)
    Σθ = [σθX^2  σθX*σθY*ρ;
          σθX*σθY*ρ  σθY^2]
    d = MvNormal([μθX, μθY], Σθ)
    return pdf(d, [x, y])
end

θXYωω (generic function with 1 method)

In [151]:
# Chapter 2: Learning

# Full learning (private + public information)
function φΞΩI(Ю, Ж)
    XΣ = Ж[2*NS+1:2*NS+Nπ]
    YΣ = Ж[2*NS+Nπ+1:2*NS+2*Nπ]

    ϵ = S .- ππ'
    ψϵ_values = ψϵω.(Ю, ϵ)
    θXY_values = θXYωω.(θXbar .- XΣ, θYbar .- YΣ)

    ΞΩI = πΩ .* ψϵ_values .* θXY_values
    ΞΩI .= ΞΩI ./ sum(ΞΩI, dims=1)

    return ΞΩI
end

φΞΩI (generic function with 1 method)

In [153]:
# Chapter 3: First-Order Conditions

# Fundamental
Δ1 = Π0
Δ2 = ππ

# Initial Wealth
W0 = e0

# FOC's
function φFOC(Ж, Ω)
    X = Ж[1:NS]
    C1 = Ж[NS+1:2*NS]
    PX, PY = Ж[2*NS+2*Nπ+1:2*NS+2*Nπ+2]

    C2_values = (W0 .- PX .* X' .- C1') ./ PY .+ X' .* Δ2
    FOCX_values = β .* sum(Ω .* exp.(-γ .* C2_values) .* (-PX ./ PY .+ Δ2), dims=1)
    FOCC1_values = exp.(-γ .* C1') + β .* sum(Ω .* exp.(-γ .* C2_values) .* (-PX ./ PY .+ Δ2), dims=1)

    return FOCX_values, FOCC1_values
end


φFOC (generic function with 1 method)

In [260]:
TEMP = [0.808868, 0.818868, 0.828868, 0.838868, 0.848868, 0.858868, 0.868868, 0.878868, 0.888868, 0.898868,
        0.908868, 0.918868, 0.928868, 0.938868, 0.948868, 0.958868, 0.968868, 0.978868, 0.988868, 0.998868,
        1.00887, 1.01887, 1.02887, 1.03887, 1.04887, 1.05887, 1.06887, 1.07887, 1.08887, 1.09887, 1.10887,
        1.11887, 1.12887, 1.13887, 1.14887, 1.15887, 1.16887, 1.17887, 1.18887, 1.19887, 1.20887, 1.21887,
        1.22887, 1.62396, 1.62647, 1.62902, 1.63159, 1.6342, 1.63684, 1.6395, 1.6422, 1.64493, 1.64769, 1.65049,
        1.65331, 1.65616, 1.65905, 1.66196, 1.66491, 1.66789, 1.67089, 1.67393, 1.677, 1.6801, 1.68323, 1.6864,
        1.68959, 1.69281, 1.69607, 1.69936, 1.70267, 1.70602, 1.7094, 1.71281, 1.71625, 1.71972, 1.72322, 1.72676,
        1.73032, 1.73391, 1.73754, 1.7412, 1.74488, 1.7486, 1.75235, 1.75613, 0.868836, 0.872748, 0.876963, 0.881499,
        0.886371, 0.891594, 0.897179, 0.903134, 0.909462, 0.916163, 0.923231, 0.930656, 0.93842, 0.946503, 0.954879,
        0.963517, 0.972385, 0.981447, 0.990664, 1., 1.00941, 1.01887, 1.02832, 1.03774, 1.04707, 1.05629, 1.06535,
        1.07422, 1.08286, 1.09123, 1.09932, 1.10708, 1.11451, 1.12157, 1.12827, 1.1346, 1.14056, 1.14614, 1.15137,
        1.15624, 1.16077, 1.16499, 1.1689, 1.08628, 1.08377, 1.08107, 1.07815, 1.07501, 1.07163, 1.06802, 1.06416,
        1.06004, 1.05567, 1.05105, 1.04618, 1.04107, 1.03574, 1.03019, 1.02445, 1.01854, 1.01248, 1.00629, 1., 0.993631,
        0.987212, 0.980768, 0.974326, 0.967915, 0.961561, 0.955293, 0.949138, 0.943122, 0.93727, 0.931606, 0.926148,
        0.920915, 0.91592, 0.911172, 0.906678, 0.90244, 0.898456, 0.894723, 0.891234, 0.88798, 0.88495, 0.882135, 0.347865,
        0.973879, 0.]


175-element Vector{Float64}:
 0.808868
 0.818868
 0.828868
 0.838868
 0.848868
 0.858868
 0.868868
 0.878868
 0.888868
 0.898868
 0.908868
 0.918868
 0.928868
 ⋮
 0.911172
 0.906678
 0.90244
 0.898456
 0.894723
 0.891234
 0.88798
 0.88495
 0.882135
 0.347865
 0.973879
 0.0

In [271]:
φFOC(TEMP,φΞΩI([0.1], TEMP))[2]

1×43 Matrix{Float64}:
 0.00150971  0.00149462  0.00147946  0.00146433  …  0.000903349  0.000889793

In [251]:
# Chapter 4: Aggregation

# Total initial wealth
ΣW0 = e0

# Aggregate Demand
function φAD(Ж, Ю)
    X = Ж[1:NS]
    C1 = Ж[NS+1:2*NS]
    XΣ = Ж[2*NS+1:2*NS+Nπ]
    YΣ = Ж[2*NS+Nπ+1:2*NS+2*Nπ]
    PX, PY = Ж[2*NS+2*Nπ+1:2*NS+2*Nπ+2]

    SΩπ = φSΩπ(Ю)

    SΩ_X_sum = sum(SΩπ .* X', dims=2)
    XiADX_values = XΣ .- SΩ_X_sum

    SΩ_C1_sum = sum(SΩπ .* C1', dims=2)
    XiADY_values = YΣ .* PY .- (ΣW0 .- PX .* XΣ .- SΩ_C1_sum)

    return XiADX_values, XiADY_values
end

# Market Clearing
function φMC(Ж, Б)
    XΣ = Ж[2*NS+1:2*NS+Nπ]
    YΣ = Ж[2*NS+Nπ+1:2*NS+2*Nπ]

    Πi = Б[1]
    θX = ΞθX[Б[2]]
    θY = ΞθY[Б[3]]

    MCX = XΣ[ΞΠi[Πi]] - (θXbar - θX)
    MCY = YΣ[ΞΠi[Πi]] - (θYbar - θY)

    return MCX, MCY
end

φMC (generic function with 1 method)

In [198]:
# Chapter 5: Zero-Info Starting Point (ξ)
W0ξ = e0

# Fixed Point Zero info
function φSYSξ(Жξ,Б)
    # Unpack parameters
    PXξ, PYξ = Жξ[1:2]
    θX = ΞθX[Б[2]]
    θY = ΞθY[Б[3]]

    # Direct solution of MC's
    Xξ = (θXbar - θX)
    C1ξ = W0ξ - PXξ * (θXbar - θX) - PYξ *(θYbar - θY)

    # Compute second-period consumption C2ξ[πi, Si]
    C2ξ_values = (W0ξ - PXξ * Xξ - C1ξ) / PYξ  .+ Xξ .* Δ2

    # FOC's
    FOCXξ_values = β .* sum(πΩ .* exp.(-γ .* C2ξ_values) .* (-PXξ ./ PYξ .+ Δ2), dims=1)
    FOCC1ξ_values = exp.(-γ .* C1ξ) .+ β .* sum(πΩ .* exp.(-γ .* C2ξ_values) .* (-1.0 ./ PYξ), dims=1)

    return vcat(FOCXξ_values, FOCC1ξ_values)
end

# Starting Point Full Code
function φЖ0(Б)
    θXi, θYi = Б[2], Б[3]
    θX = ΞθX[θXi]
    θY = ΞθY[θYi]
    solξ = nlsolve(Жξ -> φSYSξ(Жξ, [0, θXi, θYi]), [0.6, 0.7])
    PXξ, PYξ = solξ.zero
    Ж = zeros(2 * NS + 2 * Nπ + 2)
    Ж[1:NS] .= θXbar - θX
    Ж[NS+1:2*NS] .= W0ξ .- PXξ * (θXbar .- θX) .- PYξ * (θYbar .- θY)
    Ж[2*NS+2*Nπ+1:2*NS+2*Nπ+2] .= [PXξ, PYξ]
    Ж[2*NS+1:2*NS+Nπ] .= θXbar .- θX
    Ж[2*NS+Nπ+1:2*NS+2*Nπ] .= θYbar .- θY
    return Ж
end


φЖ0 (generic function with 1 method)

In [201]:
# Chapter 6: Fixed Point

# System of equations
function φSYS(Ж, Ю, Б)
    Ω = φΞΩI(Ю, Ж)
    FOCX, FOCC1 = φFOC(Ж, Ω)
    XiADX, XiADY = φAD(Ж, Ю)
    MCX, MCY = φMC(Ж, Б)

    return vcat(vec(FOCX), vec(FOCC1), vec(XiADX), vec(XiADY), MCX, MCY)
end

φSYS (generic function with 1 method)

In [259]:
Ж_ini = φЖ0([1, 3, 3])
@time for Ю_value in range(0.1, stop=0.10, length=2);
    sol = nlsolve(Ж -> φSYS(Ж, [Ю_value], [1, 3, 3]), Ж_ini); Ж_ini = sol.zero;
end

print(sol.zero')

  0.773393 seconds (4.25 M allocations: 930.190 MiB, 14.93% gc time, 13.46% compilation time)
[0.8088684269091896 0.8188684269080803 0.8288684269078905 0.8388684269070201 0.8488684269089674 0.8588684269075043 0.8688684269048907 0.878868426905025 0.8888684269049603 0.8988684269053152 0.9088684269013398 0.9188684269041779 0.9288684269016134 0.9388684269005083 0.9488684269012738 0.9588684269009576 0.9688684269001052 0.9788684268995076 0.9888684268995104 0.9988684269009103 1.0088684268977204 1.0188684268943604 1.0288684268963295 1.038868426897595 1.0488684268935182 1.0588684268933353 1.068868426896023 1.0788684268905873 1.0888684268902094 1.098868426891043 1.1088684268906928 1.1188684268902964 1.1288684268893285 1.1388684268888902 1.1488684268879388 1.1588684268848748 1.1688684268853735 1.178868426886463 1.188868426886826 1.198868426886111 1.2088684268816083 1.2188684268800436 1.2288684268838783 4.684505339814345 4.68450533912194 4.684505339683505 4.684505340302826 4.684505339677701 4.6845

In [256]:
ΞθY

-0.2:0.1:0.2



---



In [24]:
# Set a fixed seed for reproducibility
Random.seed!(1234)

# Generate a random vector Ж with mean 0.1, std 0.01, and length 177
size = 176
Ж = rand(1, size)
println("Ж = ", Ж)

Ω = φΞΩI(0.1, Ж)

Ж = [0.5798621201341324 0.4112941179498505 0.9721360824554687 0.014908849285099945 0.520354993723718 0.6395615996802734 0.8396219340580711 0.967142768915383 0.7897644095351307 0.6960406981439002 0.5667043025437501 0.5363685687293304 0.7113892493371533 0.1039294604697858 0.806704115694322 0.8705393388965176 0.9627145232215633 0.15118014494810728 0.7153546713317517 0.9395476419202511 0.5263436792092471 0.07796832278241894 0.9661966846171819 0.6665578560615069 0.33385905430793017 0.8021987716647205 0.15645840849748271 0.3807992099077905 0.06875911697436776 0.8617248852475133 0.00543655955195288 0.6984777190448724 0.3492308989506544 0.9109702293941395 0.9736533335143757 0.7606595293677095 0.4889116091598197 0.36264257055837246 0.7087378217590448 0.6566936421651552 0.1738629177690535 0.6973601779506733 0.19172328131691596 0.10051452541181105 0.4213882946411107 0.8159260441267123 0.9439639524152046 0.7392551298618548 0.01775088258636426 0.7749752377279369 0.2164667906754636 0.514119270588462

43×43 Matrix{Float64}:
 4.48661e-112  3.2615e-112   2.37089e-112  …  9.29634e-118  6.75426e-118
 5.143e-89     3.79896e-89   2.80612e-89      2.05356e-94   1.51608e-94
 2.69049e-80   2.01943e-80   1.51572e-80      2.07023e-85   1.55304e-85
 1.92324e-76   1.46683e-76   1.11871e-76      2.85179e-81   2.17385e-81
 2.42409e-80   1.87864e-80   1.4559e-80       6.92677e-85   5.36527e-85
 2.5497e-58    2.00785e-58   1.58113e-58   …  1.404e-62     1.10504e-62
 2.58558e-51   2.06895e-51   1.65552e-51      2.74369e-55   2.19429e-55
 2.07441e-47   1.68669e-47   1.37142e-47      4.24199e-51   3.44728e-51
 1.26636e-42   1.04628e-42   8.64427e-43      4.99031e-46   4.12082e-46
 9.70991e-32   8.1518e-32    6.8436e-32       7.37367e-35   6.18712e-35
 2.37117e-30   2.02278e-30   1.72555e-30   …  3.46999e-33   2.95856e-33
 2.7164e-22    2.35467e-22   2.04107e-22      7.6605e-25    6.6368e-25
 6.38612e-35   5.62499e-35   4.95449e-35      3.47054e-37   3.05526e-37
 ⋮                                       

In [25]:
Ω = φΞΩI(0.1, fill(0.1, 176))
φFOCX(fill(0.1, 176), Ω)

LoadError: UndefVarError: `φFOCX` not defined

In [26]:
function compute_PXPY_plus_e0(Ж::Vector{Float64}, Ю::Vector{Float64}, NS::Int, NΠ::Int)
    # PX is at position 2NS + 2NΠ + 1
    idx_PX = 2*NS + 2*NΠ + 1
    idx_PY = 2*NS + 2*NΠ + 2

    PX = Ж[idx_PX]
    PY = Ж[idx_PY]

    return PX * PY + e0
end

# Call the function
result = compute_PXPY_plus_e0(Ж, Ю, NS, NΠ)
println("PX × PY + e0 = ", result)


LoadError: UndefVarError: `Ю` not defined